In [1]:
# Data Collection
from nltk.corpus import gutenberg
import pandas as pd
import nltk
# Load the dataset
data=gutenberg.raw('shakespeare-hamlet.txt')

# save to a file
with open('hamlet.txt','w') as file:
    file.write(data)


In [1]:
# Data Preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer #Will help to convert our text into vectors
from tensorflow.keras.preprocessing.sequence import pad_sequences# Makes sure that all sentence length will be same 
from sklearn.model_selection import train_test_split

In [ ]:
# Load the dataset
with open('hamlet.txt','r') as file:
    text=file.read().lower()

In [ ]:
# Tokenize the text-creating indexes for words
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])#Applies tokenization on the text that we are giving
total_words=len(tokenizer.word_index)+1


In [ ]:
# create input sequences
input_sequences=[]
for line in text.split('\n'):
    print(line)
    token_list=tokenizer.texts_to_sequences([line])[0]
    print(token_list)
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        print(n_gram_sequence)
        input_sequences.append(n_gram_sequence)

[the tragedie of hamlet by william shakespeare 1599]
[1, 687, 4, 45, 41, 1886, 1887, 1888]
[1, 687]
[1, 687, 4]
[1, 687, 4, 45]
[1, 687, 4, 45, 41]
[1, 687, 4, 45, 41, 1886]
[1, 687, 4, 45, 41, 1886, 1887]
[1, 687, 4, 45, 41, 1886, 1887, 1888]

[]

[]
actus primus. scoena prima.
[1180, 1889, 1890, 1891]
[1180, 1889]
[1180, 1889, 1890]
[1180, 1889, 1890, 1891]

[]
enter barnardo and francisco two centinels.
[57, 407, 2, 1181, 177, 1892]
[57, 407]
[57, 407, 2]
[57, 407, 2, 1181]
[57, 407, 2, 1181, 177]
[57, 407, 2, 1181, 177, 1892]

[]
  barnardo. who's there?
[407, 1182, 63]
[407, 1182]
[407, 1182, 63]
  fran. nay answer me: stand & vnfold
[408, 162, 377, 21, 247, 882]
[408, 162]
[408, 162, 377]
[408, 162, 377, 21]
[408, 162, 377, 21, 247]
[408, 162, 377, 21, 247, 882]
your selfe
[18, 66]
[18, 66]

[]
   bar. long liue the king
[451, 224, 248, 1, 30]
[451, 224]
[451, 224, 248]
[451, 224, 248, 1]
[451, 224, 248, 1, 30]

[]
   fran. barnardo?
[408, 407]
[408, 407]
  bar. he
[451, 25]
[451

In [ ]:
# Pad Sequences
max_sequence_len=max([len(x) for x in input_sequences])
print(max_sequence_len)

14


In [ ]:
# fit_on_texts()-->Builds a word index (dictionary)
# → mapping each word to a unique integer ID.
# Stores that mapping inside the tokenizer object.

# texts_to_sequences()-->This uses the word index that was built by fit_on_texts() to convert text into integer sequences.

In [ ]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
# Make all the sentences to 14 indexes
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [ ]:
# Create predictors and label
import tensorflow as tf
X,y=input_sequences[:,:-1],input_sequences[:,-1]

In [ ]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)#to_categorical() converts those integer labels into one-hot encoded vectors.

y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
# Split the data into train and test sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
X_train

array([[   0,    0,    0, ...,    0,  116,   62],
       [   0,    0,    0, ...,    0,   12,  168],
       [   0,    0,    0, ...,  164,  119, 2630],
       ...,
       [   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    0,    7],
       [   0,    0,    0, ...,   11,   67,   87]])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(monitor='val_loss',patience=60,restore_best_weights=True)


In [ ]:
# Train our LSTM RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,LSTM,Dropout
# Dropout is used to deactviate some neurons randomly to prevent overfitting

model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))
# Compile the model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 13, 100)           481800    
                                                                 
 lstm (LSTM)                 (None, 13, 150)           150600    
                                                                 
 dropout (Dropout)           (None, 13, 150)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train the model
history=model.fit(
    X_train,y_train,epochs=50,validation_data=(X_test,y_test),
    verbose=1,
    
)

Epoch 1/50


644/644 [==============================] - 35s 46ms/step - loss: 6.8987 - accuracy: 0.0333 - val_loss: 6.7555 - val_accuracy: 0.0352
Epoch 2/50
644/644 [==============================] - 24s 38ms/step - loss: 6.4660 - accuracy: 0.0370 - val_loss: 6.8589 - val_accuracy: 0.0439
Epoch 3/50
644/644 [==============================] - 22s 34ms/step - loss: 6.3300 - accuracy: 0.0435 - val_loss: 6.9118 - val_accuracy: 0.0492
Epoch 4/50
644/644 [==============================] - 24s 37ms/step - loss: 6.1898 - accuracy: 0.0506 - val_loss: 6.9242 - val_accuracy: 0.0484
Epoch 5/50
644/644 [==============================] - 20s 30ms/step - loss: 6.0451 - accuracy: 0.0561 - val_loss: 6.9520 - val_accuracy: 0.0560
Epoch 6/50
644/644 [==============================] - 20s 30ms/step - loss: 5.9071 - accuracy: 0.0625 - val_loss: 6.9770 - val_accuracy: 0.0558
Epoch 7/50
644/644 [==============================] - 19s 30ms/step - loss: 5.7814 - accuracy: 0.0678 - val_loss: 7.0359 - val_accurac

In [ ]:
# Funcition to predict the next word
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list=token_list[-(max_sequence_len-1):]#Ensure that the sequence length matches max_sequence_len - 1
        # Here we drop the older token and keep only the most recent ones
    token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted,axis=1)[0]
    for word,index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word
    return None


In [ ]:
input_text="To be or not to"
print(f"Input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
print(max_sequence_len)
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next word:{next_word}")

Input text:To be or not to
14
Next word:perseuer


In [ ]:
# Save the model
model.save('next_word_model.h5')

# save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
   pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

c:\Users\Tridib Roy Chowdhury\OneDrive\Desktop\CODING\Next-Word-Prediction-Project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
